# Basic dynamic model

In [1]:
DATA_PATH = '../backend/data/dynamic_dataset'
DATA_PATH = '../backend/dynamic_signs/frames'


In [2]:
#Load DATA
from sign.training.landmark_extraction.MediaPiper import MediaPiper

mp = MediaPiper()
gestures = mp.process_dynamic_gestures_from_folder(DATA_PATH)

2024-03-14 11:36:06.529365: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-14 11:36:06.532819: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-14 11:36:06.571688: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 11:36:06.571717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 11:36:06.572577: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
[gesture.label for gesture in gestures]

['J', 'Z']

In [4]:
import numpy as np
from sign.trajectory import TrajectoryBuilder, trajectory

bob = TrajectoryBuilder()

gesture_trajector_map: dict[str, list[trajectory]] = {}

for gesture in gestures:
    gesture_trajector_map[gesture.label] = []
    for sequence in gesture.results:
        new_sequence = []
        for image_mp_res in sequence:
            hand_landmarks = np.array(image_mp_res.multi_hand_landmarks)
            new_sequence.append(hand_landmarks)
        new_sequence = bob.extract_keyframes_sample(new_sequence)
        sequence_as_np_array = np.array(new_sequence)
        sequence_trajectory = bob.make_trajectory(sequence_as_np_array)

        gesture_trajector_map[gesture.label].append(sequence_trajectory)


print(gesture_trajector_map)

🔥🔥 TrajectoryBuilder is now running in BERTRAM_MODE 🔥🔥
{'J': [trajectory(directions=[trajectory_element(x=<direction.UP: 1>, y=<direction.STATIONARY: 0>, z=<direction.DOWN: -1>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.UP: 1>, z=<direction.STATIONARY: 0>)]), trajectory(directions=[trajectory_element(x=<direction.UP: 1>, y=<direction.UP: 1>, z=<direction.DOWN: -1>), trajectory_element(x=<direction.DOWN: -1>, y=<direction.DOWN: -1>, z=<direction.UP: 1>)]), trajectory(directions=[trajectory_element(x=<direction.DOWN: -1>, y=<direction.UP: 1>, z=<direction.DOWN: -1>), trajectory_element(x=<direction.DOWN: -1>, y=<direction.DOWN: -1>, z=<direction.UP: 1>)]), trajectory(directions=[trajectory_element(x=<direction.DOWN: -1>, y=<direction.UP: 1>, z=<direction.DOWN: -1>), trajectory_element(x=<direction.DOWN: -1>, y=<direction.DOWN: -1>, z=<direction.UP: 1>)])], 'Z': [trajectory(directions=[trajectory_element(x=<direction.UP: 1>, y=<direction.UP: 1>, z=<direction.STATIONARY

In [5]:
X = []
y = []

for k,gesture in gesture_trajector_map.items():
    min = len(gesture[0].directions)
    for trajectory in gesture[1:]:
        length = len(trajectory.directions)
        if min > length:
            min = length
    for trajectory in gesture:
        if len(trajectory.directions) > min:
            for i in range(len(trajectory.directions) - min):
                trajectory.directions.pop()


    for trajector in gesture:
        xyz  = []
        for te in trajector.directions:
            xyz.append(te.x.value)
            xyz.append(te.y.value)
            xyz.append(te.z.value)
        X.append(np.array(xyz))
        y.append(k)

X = np.array(X)
y = np.array(y)
X

array([[ 1,  0, -1,  0,  1,  0],
       [ 1,  1, -1, -1, -1,  1],
       [-1,  1, -1, -1, -1,  1],
       [-1,  1, -1, -1, -1,  1],
       [ 1,  1,  0, -1,  1,  0],
       [ 1,  1,  0,  1,  1, -1],
       [ 1,  1,  0,  1,  0,  0]])

In [6]:
from sklearn.linear_model import LinearRegression, LogisticRegression

model = LogisticRegression()
model.fit(X,y)

LogisticRegression()

In [7]:
from joblib import dump
dump(model, 'dynamic_model.joblib')

['dynamic_model.joblib']